In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from uszipcode import SearchEngine

In [2]:
df = pd.read_csv('data/rollingsales_bronx.csv',sep=';')

In [3]:
df.columns[df.isna().any()].tolist()
print(df.columns)

Index([u'BOROUGH', u'NEIGHBORHOOD', u'BUILDING CLASS CATEGORY',
       u'TAX CLASS AT PRESENT', u'BLOCK', u'LOT', u'EASE-MENT',
       u'BUILDING CLASS AT PRESENT', u'ADDRESS', u'APARTMENT NUMBER',
       u'ZIP CODE', u'RESIDENTIAL UNITS', u'COMMERCIAL UNITS', u'TOTAL UNITS',
       u'LAND SQUARE FEET', u'GROSS SQUARE FEET', u'YEAR BUILT',
       u'TAX CLASS AT TIME OF SALE', u'BUILDING CLASS AT TIME OF SALE',
       u' SALE PRICE ', u'SALE DATE'],
      dtype='object')


In [4]:
df.columns = df.columns.str.replace(' ','_')
df.columns

Index([u'BOROUGH', u'NEIGHBORHOOD', u'BUILDING_CLASS_CATEGORY',
       u'TAX_CLASS_AT_PRESENT', u'BLOCK', u'LOT', u'EASE-MENT',
       u'BUILDING_CLASS_AT_PRESENT', u'ADDRESS', u'APARTMENT_NUMBER',
       u'ZIP_CODE', u'RESIDENTIAL_UNITS', u'COMMERCIAL_UNITS', u'TOTAL_UNITS',
       u'LAND_SQUARE_FEET', u'GROSS_SQUARE_FEET', u'YEAR_BUILT',
       u'TAX_CLASS_AT_TIME_OF_SALE', u'BUILDING_CLASS_AT_TIME_OF_SALE',
       u'_SALE_PRICE_', u'SALE_DATE'],
      dtype='object')

In [5]:
#Dropping very sparse columns with potentially little info
df = df.drop(['EASE-MENT','APARTMENT_NUMBER'],axis=1)

In [6]:
df['SALE_DATE']

0       17/08/2018
1       22/03/2018
2       06/11/2017
3       05/03/2018
4       13/11/2017
5       22/02/2018
6       15/02/2018
7       18/07/2018
8       23/02/2018
9       22/03/2018
10      19/06/2018
11      22/08/2018
12      20/03/2018
13      26/03/2018
14      19/01/2018
15      28/09/2018
16      30/05/2018
17      28/06/2018
18      05/06/2018
19      20/12/2017
20      13/11/2017
21      23/07/2018
22      11/04/2018
23      29/08/2018
24      28/08/2018
25      29/08/2018
26      07/08/2018
27      03/01/2018
28      18/07/2018
29      12/04/2018
           ...    
7620    28/11/2017
7621    30/11/2017
7622    30/04/2018
7623    30/07/2018
7624    13/08/2018
7625    20/09/2018
7626    15/10/2018
7627    27/12/2017
7628    10/05/2018
7629    20/08/2018
7630    23/08/2018
7631    04/09/2018
7632    16/11/2017
7633    23/05/2018
7634    20/06/2018
7635    06/02/2018
7636    03/04/2018
7637    25/07/2018
7638    30/07/2018
7639    04/09/2018
7640    13/09/2018
7641    15/1

In [7]:
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'],format='%d/%m/%Y')
df['_SALE_PRICE_'] = df['_SALE_PRICE_'].apply(lambda x:x.replace(' ','').replace('$',''))
df['year'] = df['SALE_DATE'].apply(lambda x:x.year)
df['month'] = df['SALE_DATE'].apply(lambda x:x.month)
df['weekday'] = df['SALE_DATE'].apply(lambda x:x.weekday())

In [8]:
def clean_df(filename):
    
    df = pd.read_csv('data/'+filename,sep=';')
    df.columns = df.columns.str.replace(' ','_')
    #Dropping very sparse columns
    df = df.drop(['EASE-MENT','APARTMENT_NUMBER'],axis=1)
    
    #Dropping houses with missing price
    df = df[df['_SALE_PRICE_'] != " $-   "]
        
    #Turn Sale price column in numerical data
    df['_SALE_PRICE_'] = df['_SALE_PRICE_'].apply(lambda x:int(x.replace(' ','').replace('$','')))
    df['_SALE_PRICE_']=df['_SALE_PRICE_'].astype('int64')
    
    #Extract year/month/weekday and drop data column
    df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'],format='%d/%m/%Y')
    
    #We keep year as a numerical feature (maybe not enough data to use as categorical)
    df['year'] = df['SALE_DATE'].apply(lambda x:x.year)
    
    #Month and weekday as categorical
    df['month'] = df['SALE_DATE'].apply(lambda x:x.month).astype('category')
    df['weekday'] = df['SALE_DATE'].apply(lambda x:x.weekday()).astype('category')
    
    df['TAX_CLASS_AT_TIME_OF_SALE'] = df['TAX_CLASS_AT_TIME_OF_SALE'].astype('category')
    
    df = df.drop(['SALE_DATE'],axis=1)
    df = df.drop(['ADDRESS'],axis=1)
    
    return df
    

In [9]:
df1 = clean_df('rollingsales_bronx.csv')
df2 = clean_df('rollingsales_brooklyn.csv')
df3 = clean_df('rollingsales_manhattan.csv')
df4 = clean_df('rollingsales_queens.csv')
df5 = clean_df('rollingsales_statenisland.csv')

df=pd.concat([df1,df2,df3,df4,df5])
df.shape

(57685, 20)

In [10]:
print(df._get_numeric_data().columns)
print(df.shape)

Index([u'BOROUGH', u'BLOCK', u'LOT', u'ZIP_CODE', u'RESIDENTIAL_UNITS',
       u'COMMERCIAL_UNITS', u'TOTAL_UNITS', u'LAND_SQUARE_FEET',
       u'GROSS_SQUARE_FEET', u'YEAR_BUILT', u'_SALE_PRICE_', u'year'],
      dtype='object')
(57685, 20)


In [11]:
#600 columns with NaN values in the coordinates features
df = df.dropna()
df.shape
#col = df._get_numeric_data().columns.tolist()
#col.remove('_SALE_PRICE_')

(57685, 20)

In [12]:
scaler = StandardScaler()

df[col] = scaler.fit_transform(df[col]) 

print(df.shape)


NameError: name 'col' is not defined

In [ ]:
df = pd.get_dummies(df)
print(df.shape)

In [ ]:
df.to_csv('data/merged_df_base.csv')